# Vehicle dynamics

The position of vehicle $i$ is $x_i(t)$ The distance to the vehicle or object
in front of vehicle $i$ is $q_i(t)$.

The vehicle speed $v_i(t) = \dot{x}_i(t)$. 

The control input to the vehicle is the acceleration $u_i$.

The desired distance to the obstacle in front $\hat{q}_i$ is calculated by scaling
with the current vehicle speed:

$$ \hat{q}_i(t) = k_q v_i(t) $$

Define a PD controller as:

$$ r_i(t) = q_i(t) - \hat{q}_i(t) $$
$$ u_i = k_p r_i(t) + k_d \frac{r_i(t) - r_i(t-\Delta t)}{\Delta t} $$


## Single vehicle approaching fixed obstacle

Let there be a vehicle starting at $x=0$ with an obstacle at $200$ so $q=200-x$.

In [1]:
%matplotlib qt
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.transforms import Affine2D
from matplotlib.collections import CircleCollection


In [38]:
class RandomTraffic:
    def __init__(self, 
        n_vehicles: int,
        track_length: float,
        k_p: float, 
        k_d: float,
        k_q: float,
        k_qmin: float,
        v_min: float,
        v_max: float
    ) -> None:
        self.n_vehicles = n_vehicles
        self.track_length = track_length
        self.k_p = k_p
        self.k_d = k_d
        self.k_q = k_q
        self.k_qmin = k_qmin
        self.v_min = v_min
        self.v_max = v_max
        self.position = np.zeros(n_vehicles, dtype=np.float32)
        self.velocity = np.zeros(n_vehicles, dtype=np.float32)
        self.active = np.zeros(n_vehicles, dtype=np.bool8)
        self.r_old = np.zeros(n_vehicles, dtype=np.float32)
    
    def enable_vehicle(self):
        """Find the first available vehicle slot and set active to True"""
        argmin = self.active.argmin()
        
        if not self.active[argmin]:
            # If argmin is already active, there's no space to add another vehicle
            self.active[argmin] = True
            self.position[argmin] = 0.0
            self.velocity[argmin] = 0.0

    def q(self):
        p = self.position[self.active]
        distance = np.zeros_like(p)

        order = np.argsort(p)
        diff = np.diff(p[order])
        distance[order] = np.concatenate([diff, [100]])
        output = np.zeros_like(self.position)
        output[self.active] = distance        
        return output

    def qhat(self):
        return np.max([self.k_qmin * np.ones_like(self.velocity), self.k_q * self.velocity], axis=0)

    def r(self):
        return self.q() - self.qhat()

    def r_diff(self, dt):
        return (self.r() - self.r_old) / dt

    def u(self, dt):
        return self.k_p * self.r() + self.k_d * self.r_diff(dt)
        
    def step(self, dt: float):
        u_calc = self.u(dt)
        v_calc = self.velocity + dt * u_calc
        v_calc = np.clip(v_calc, self.v_min, self.v_max)
        x_calc = self.position + dt * v_calc

        self.velocity = v_calc
        self.position = x_calc

        self.active = self.active * (self.position < self.track_length)

        self.r_old = self.r()
        

In [39]:
class Circles:
    def __init__(self, traffic: RandomTraffic):
        self.traffic = traffic
        self.fig = plt.figure(figsize=(5, 1))
        self.ax = self.fig.add_axes([0, 0, 1, 1], frameon=False)
        self.ax.set_xlim([0, 200])
        self.ax.set_ylim([0, 20])

        self.circles = CircleCollection([], animated=True)
        self.ax.add_artist(self.circles)

    def update(self):
        positions = self.traffic.position[self.traffic.active]
        self.circles.set_sizes(np.ones_like(positions) * 10)
        self.circles.set_offsets(np.c_[positions, 50 * np.ones_like(positions)])
        return [self.circles]


In [40]:
t = RandomTraffic(
    n_vehicles=100,
    track_length=200,
    k_p=5,
    k_d=1,
    k_q=1,
    k_qmin=4,
    v_min=0,
    v_max=20
)
t.enable_vehicle()
circles = Circles(t)

def update(frame):
    if np.random.rand() < 0.05:
        t.enable_vehicle()
    t.step(0.05)
    return circles.update()

anim = FuncAnimation(circles.fig, update, interval=50, blit=True)


In [44]:
t.q()

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  17.275097  ,  12.903612  ,
         7.0669556 ,   0.22277236,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
        19.531563  ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [15]:
position = np.array([4.0,3.5,2,1.4,0])
active = np.array([False, True, True, False, False])
output = np.zeros_like(position)
d = -np.diff(position[active])
output[active] = np.concatenate([[100], d])
output

array([  0. , 100. ,   1.5,   0. ,   0. ])

In [53]:
position = np.array([0.0, 0.0, 0.0, 12, 0.0, 3.4])
active = np.array([True, True, True, False, False, False])

p = position[active]
out = np.zeros_like(p)

order = np.argsort(p)
diff = np.diff(p[order])
out[order] = np.concatenate([diff, [100]])
output = np.zeros_like(position)
output[active] = out

In [54]:
output

array([  0.,   0., 100.,   0.,   0.,   0.])